In [3]:
import numpy as np
import math
import scipy.special
import pprint

class LinearNeuralNetwork:
    
    def __init__(self, learningrate: float, inputnodes: int, outputnodes: int, hiddennodes_list: list = None):
        '''Create neural network'''
        
        # create list that contains nodes count for every layer
        
        # if None - NN doesnt contains hidden layers
        if hiddennodes_list is None or len(hiddennodes_list) == 0:
            layers_list = [inputnodes, outputnodes]
        else:
            layers_list = [inputnodes] + hiddennodes_list + [outputnodes]
        
        # create weights matrix for every layer
        self._weights = []
        
        # matrixs count = layers count - 1
        # matrix = [row] x [column] = [next layer] x [previous layer] => range(len(layers_list) - 1)
        
        for i in range(len(layers_list) - 1):
            self._weights.append(np.random.normal(0.0, math.pow(layers_list[i + 1], -0.5), 
                                                  (layers_list[i + 1], layers_list[i])))
        # learning rate
        self._lr = learningrate
        
        # activation function
        self._act_func = lambda x: scipy.special.expit(x)
    
    def train(self, inputs_list, targets_list):
        
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # COLLECT ALL OUTPUTS
        
        output_column = inputs
        outputs_list = [inputs]
        
        for i in range(len(self._weights)):
            if i != len(self._weights) - 1:
                # apply sigmoid function
                output_column = self._act_func(np.dot(self._weights[i], output_column))
                outputs_list.append(output_column)
            else:
                # apply linear function
                output_column = np.dot(self._weights[i], output_column)
                outputs_list.append(output_column)
        
        # COLLECT ALL ERRORS
        
        output_errors = targets - output_column
        errors_list = [output_errors]
        
        for wm in self._weights[:0:-1]:
            output_errors = np.dot(wm.T, output_errors)
            errors_list.insert(0, output_errors)
        
        # IMPROVE WEIGHTS IN MATRIX
        
        # сделал так, чтобы ничего не сломать
        for i in range(len(self._weights)):
            if i != len(self._weights) - 1:
                self._weights[i] += self._lr * np.dot(errors_list[i] * outputs_list[i+1] * (1.0 - outputs_list[i+1]), 
                                                      outputs_list[i].T)      
            else:
                self._weights[i] += self._lr * np.dot(errors_list[i], outputs_list[i].T)
    
    def query(self, inputs):
        '''Calculate result with inputs values'''
        
        # transform row in column to multiple on matrix
        res_column = np.array(inputs, ndmin=2).T
        
        # by last layer i apply linear function
        for i in range(len(self._weights) - 1):
            res_column = self._act_func(np.dot(self._weights[i], res_column))
        
        # apply linear function
        res_column = np.dot(self._weights[-1], res_column)
        
        return res_column        
    
    def print_layers(self):
        
        for wm in self._weights:
            pprint.pprint(wm)
            
    @property
    def weights(self):
        #
        return self._weights
    
    @weights.setter
    def weights(self, weights):
        #
        self._weights = weights
            

In [4]:
nn = LinearNeuralNetwork(0.1, 2, 1, [])
nn.print_layers()

array([[-1.74377709,  0.0043588 ]])


In [233]:
nn.query([1, 2])

array([[-0.3]])